In [28]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

Cloning into 'FontDiffusion'...
remote: Enumerating objects: 20748, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 20748 (delta 58), reused 64 (delta 30), pack-reused 20647 (from 2)
Receiving objects: 100% (20748/20748), 278.12 MiB | 33.15 MiB/s, done.
Resolving deltas: 100% (1054/1054), done.
Updating files: 100% (138/138), done.


In [2]:
import os
import sys
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"
    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name

def load_secret(key_name: str) -> Optional[str]:
    env = ENV_NAME
    secret_value = None
    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")
    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else:
            secret_value = os.getenv(key_name)
        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None
        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value
    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None

def print_system_info():
    print("\n🔧 System Information")
    print(f"Python version: {sys.version.split()[0]}")
    try:
        import torch
        print(f"PyTorch version: {torch.__version__}")
        if torch.cuda.is_available():
            print(f"CUDA version: {torch.version.cuda}")
            print(f"GPU count: {torch.cuda.device_count()}")
            for i in range(torch.cuda.device_count()):
                print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        else:
            print("CUDA not available")
    except ImportError:
        print("PyTorch not installed")
    finally:
      !nvidia-smi

INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()
is_kaggle = ("kaggle" in ENV_NAME)
is_colab = not is_kaggle
print_system_info()

os.environ["WANDB_API_KEY"] = wandb_key = load_secret("WANDB_API_KEY")
os.environ["HF_TOKEN"] = HF_TOKEN = load_secret('HF_TOKEN')
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Now, these libraries will log in automatically
import wandb
import huggingface_hub

wandb.login()
huggingface_hub.login(token=os.environ["HF_TOKEN"])

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/

🔧 System Information
Python version: 3.12.12
PyTorch version: 2.9.0+cu126
CUDA version: 12.6
GPU count: 1
  GPU 0: Tesla T4
Sat Jan  3 03:09:01 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             1

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: dungngocpham171 (dungngocpham171-university-of-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
!uv pip install --upgrade pip
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!uv pip install torch==2.9 torchvision
# 4. Install other dependencies

print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
# !sudo apt-get update && sudo apt-get install dos2unix
!uv pip install gdown tqdm
!uv pip install wandb hf_transfer
!uv pip install --upgrade pyarrow datasets
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 112ms
Prepared 1 package in 92ms
Uninstalled 1 package in 39ms
Installed 1 package in 19ms
 - pip==24.1.2
 + pip==25.3
/content
Using Python 3.12.12 environment at: /usr
Audited 2 packages in 154ms

⬇️ Installing Dependencies (Manually fixed)...
  × No solution found when resolving dependencies:
  ╰─▶ Because torch==1.13.1 has no wheels with a matching Python ABI tag
      (e.g., `cp312`) and xformers==0.0.16 depends on torch==1.13.1, we can
      conclude that xformers==0.0.16 cannot be used.
      And because you require xformers==0.0.16, we can conclude that your
      requirements are unsatisfiable.

      hint: You require CPython 3.12 (`cp312`), but we only found wheels for
      `torch` (v1.13.1) with the following Python ABI tags: `cp37m`, `cp38`,
      `cp39`, `cp310`, `cp311`
Using Python 3.12.12 environment at: /usr
Resolved 47 packages in 324ms
  × Failed to build `tokenizers==0.13.3`
  ├─▶ The build backend re

In [4]:
# KAGGLE CELL #1: Download checkpoint
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2" "protobuf<5.0.0" "numpy<2.0.0"
import os
import sys
from tqdm.auto import tqdm
from pathlib import Path
os.chdir(OUTPUT_PATH)
import os
from pathlib import Path
def download_from_hf(
    repo_id: str,
    local_dir: str = "ckpt",
    allow_patterns=None,
    force_download: bool = False,
    repo_type: str = "model"
):
    if allow_patterns is None:
        allow_patterns = ["*.safetensors", "scr*"]
    print(f"📥 Downloading checkpoint from Hugging Face Hub to '{local_dir}'...\n")
    from huggingface_hub import snapshot_download
    snapshot_download(
        repo_id=repo_id,
        local_dir=local_dir,
        repo_type=repo_type,
        allow_patterns=allow_patterns,
        force_download=force_download
    )
    print("\n✅ Download complete!")
    print(f"\n📂 Files in {local_dir}/:")
    for file in os.listdir(local_dir):
        if file.endswith(".safetensors"):
            size = os.path.getsize(os.path.join(local_dir, file)) / (1024**2)
            print(f"  ✓ {file} ({size:.2f} MB)")

Using Python 3.12.12 environment at: /usr
Resolved 13 packages in 95ms
Prepared 2 packages in 56ms
Uninstalled 2 packages in 3ms
Installed 2 packages in 5ms
 - fsspec==2025.10.0
 + fsspec==2025.12.0
 - huggingface-hub==1.2.3
 + huggingface-hub==0.36.0


In [29]:
download_from_hf(
    repo_id="dzungpham/font-diffusion-weights",
    local_dir="ckpt",
    allow_patterns=["content*", "style*", "unet*", "scr*"]
)

📥 Downloading checkpoint from Hugging Face Hub to 'ckpt'...



content_encoder.pth:   0%|          | 0.00/4.77M [00:00<?, ?B/s]

style_encoder.pth:   0%|          | 0.00/82.4M [00:00<?, ?B/s]

unet.pth:   0%|          | 0.00/315M [00:00<?, ?B/s]


✅ Download complete!

📂 Files in ckpt/:
  ✓ style_encoder.safetensors (78.58 MB)
  ✓ content_encoder.safetensors (4.54 MB)
  ✓ unet.safetensors (300.34 MB)
  ✓ total_model.safetensors (383.47 MB)


In [6]:
download_from_hf(
    repo_id="dzungpham/font-diffusion-generated-data",
    local_dir="NomTuTao",
    repo_type="dataset",
    allow_patterns=["*Nom*"]
)

📥 Downloading checkpoint from Hugging Face Hub to 'NomTuTao'...



Ds_10k_ChuNom_TuTao.txt: 0.00B [00:00, ?B/s]

Ds_300_ChuNom_TuTao.csv: 0.00B [00:00, ?B/s]

Top184_PureNomChar.xlsx:   0%|          | 0.00/15.8k [00:00<?, ?B/s]


✅ Download complete!

📂 Files in NomTuTao/:


In [7]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /content/.


In [8]:
print("Model files:")
!ls -larth {OUTPUT_PATH}/ckpt

Model files:
total 1.1G
drwxr-xr-x 3 root root 4.0K Jan  3 03:09 .cache
-rw-r--r-- 1 root root 4.6M Jan  3 03:09 content_encoder.safetensors
drwxr-xr-x 2 root root 4.0K Jan  3 03:09 global_step_250
drwxr-xr-x 2 root root 4.0K Jan  3 03:09 global_step_500
-rw-r--r-- 1 root root 272M Jan  3 03:09 scr_210000.pth
-rw-r--r-- 1 root root  79M Jan  3 03:09 style_encoder.safetensors
-rw-r--r-- 1 root root 384M Jan  3 03:10 total_model.safetensors
-rw-r--r-- 1 root root 301M Jan  3 03:10 unet.safetensors
drwxr-xr-x 5 root root 4.0K Jan  3 03:10 .
drwxr-xr-x 1 root root 4.0K Jan  3 03:10 ..


In [9]:
# @title Exporting train original data from HF
%cd {OUTPUT_PATH}
HF_USERNAME = "dzungpham"
# Train Split
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output-dir "my_dataset/train_original" \
  --repo-id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train_original" \
  --token HF_TOKEN

/content
2026-01-03 03:10:25,667 - __main__ - INFO - Starting dataset export...
2026-01-03 03:10:25,667 - __main__ - INFO - Loading dataset from Hub: dzungpham/font-diffusion-generated-data (split: train_original)
README.md: 3.05kB [00:00, 14.8MB/s]
data/train_original-00000-of-00001.parqu(…): 100% 146M/146M [00:02<00:00, 49.2MB/s]
data/train-00000-of-00001.parquet: 100% 93.4M/93.4M [00:02<00:00, 41.2MB/s]
data/val-00000-of-00001.parquet: 100% 4.36M/4.36M [00:02<00:00, 1.89MB/s]
Generating train_original split: 100% 10865/10865 [00:00<00:00, 15859.18 examples/s]
Generating train split: 100% 6924/6924 [00:00<00:00, 16018.15 examples/s]
Generating val split: 100% 442/442 [00:00<00:00, 20461.60 examples/s]
2026-01-03 03:10:39,274 - __main__ - INFO - Loaded 10865 samples from Hub
2026-01-03 03:10:39,274 - __main__ - INFO - Created directory structure at my_dataset/train_original
2026-01-03 03:10:39,274 - __main__ - INFO - Exporting images...
Exporting images: 100% 10865/10865 [00:27<00:00,

In [ ]:
# @title Exporting train data from HF
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output-dir "my_dataset/train" \
  --repo-id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train" \
  --token HF_TOKEN

In [ ]:
# @title Exporting validation data from HF
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output-dir "my_dataset/val" \
  --repo-id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val" \
  --token HF_TOKEN

In [10]:
# @title Show fonts and styles images
print("Fonts currently in fonts/ folder")
!ls -lt FontDiffusion/fonts
print("Styles in style_images/ folder")
!ls -l FontDiffusion/styles_images

Fonts currently in fonts/ folder
total 332584
-rw-r--r-- 1 root root 26929728 Jan  3 03:08  NomNaTongLight2.ttf
-rw-r--r-- 1 root root 14574480 Jan  3 03:08  NomNaTongLight.ttf
-rw-r--r-- 1 root root 31729820 Jan  3 03:08  NomNaTong-Regular2.otf
-rw-r--r-- 1 root root 14574552 Jan  3 03:08  NomNaTong-Regular.ttf
-rw-r--r-- 1 root root  9424552 Jan  3 03:08  NomNaTong-Regular.otf
-rw-r--r-- 1 root root 12967288 Jan  3 03:08  HanaMinC.otf
-rw-r--r-- 1 root root 30739236 Jan  3 03:08  HanaMinB.ttf
-rw-r--r-- 1 root root 32201032 Jan  3 03:08  HanaMinB.otf
-rw-r--r-- 1 root root 22761228 Jan  3 03:08  HanaMinA.ttf
-rw-r--r-- 1 root root 31621108 Jan  3 03:08  HanaMinA.otf
-rw-r--r-- 1 root root 18202176 Jan  3 03:08 'Han-nom Minh 1.42.otf'
-rw-r--r-- 1 root root 19505228 Jan  3 03:08  Han-Nom-Khai-Regular-300623.ttf
-rw-r--r-- 1 root root 20368044 Jan  3 03:08 'Han-Nom Kai 1.00.otf'
-rw-r--r-- 1 root root 33815824 Jan  3 03:08 'HAN NOM B.ttf'
-rw-r--r-- 1 root root 21320444 Jan  3 03:08 'H

In [30]:
# @title Run batch generation
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2" "protobuf<5.0.0" "numpy<2.0.0"
%cd {OUTPUT_PATH}
# No need to pass num_processes because accelerate auto detect num gpus on machine
!accelerate launch FontDiffusion/sample_batch_multi_gpus.py \
    --characters "NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts/NomNaTong-Regular.otf" \
    --output_dir "my_dataset/train_original" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --start_line 900 \
    --end_line 950 \
    --batch_size 35 \
    --save_interval 1 \
    --channels_last \
    --seed 42 \
    --compile \
    --enable_xformers

Using Python 3.12.12 environment at: /usr
Resolved 13 packages in 50ms
Audited 13 packages in 0.35ms
/content
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/content/FontDiffusion/src/dpm_solver/dpm_solver_pytorch.py:53: SyntaxWarning: invalid escape sequence '\h'
  The `alphas_cumprod` is the \hat{alpha_n} arrays in the notations of DDPM. Specifically, DDPMs assume that
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
📖 Reading character file: 100%|██████████████████████████| 51.0/51.0 [00:00<00:00, 164kiteration/s]
✓ Verifying style images: 100%|██████████████████████████| 15.0/15.0 [00:00<00:00, 46.9kiteration/s]
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855

In [19]:
# @title Count images in ContentImage and TargetImage
!find my_dataset/train_original/ContentImage -type f | wc -l
!find my_dataset/train_original/TargetImage -type f | wc -l

899
13485


In [31]:
# @title Train Validation split
!python FontDiffusion/create_validation_split.py \
  --data_root my_dataset \
  --val_ratio 0.2 \
  --seed 42

2026-01-03 05:37:10,060 | INFO | ✓ Using source directory: my_dataset/train_original
2026-01-03 05:37:10,060 | INFO | 
2026-01-03 05:37:10,060 | INFO | FONTDIFFUSION VALIDATION SPLIT CREATOR
2026-01-03 05:37:10,061 | INFO | ============================================================
2026-01-03 05:37:10,061 | INFO | 
2026-01-03 05:37:10,061 | INFO | ANALYZING TRAINING DATA
2026-01-03 05:37:10,061 | INFO | ============================================================
2026-01-03 05:37:10,061 | INFO | 
🔍 Scanning content images...
Content images: 100%|████████████████████████████████████████████| 948/948 [00:00<00:00, 1.04Mimg/s]
2026-01-03 05:37:10,066 | INFO |   ✓ Found 948 content images
2026-01-03 05:37:10,066 | INFO | 
🔍 Scanning target images...
Styles: 100%|██████████████████████████████████████████████████| 15.0/15.0 [00:00<00:00, 198style/s]
2026-01-03 05:37:10,143 | INFO |   ✓ Found 14220 valid target images
2026-01-03 05:37:10,143 | INFO | 
🔍 Validating content ↔ target pairs...

In [21]:
!uv pip install --upgrade pyarrow datasets

Using Python 3.12.12 environment at: /usr
Resolved 39 packages in 257ms
Prepared 2 packages in 0.58ms
Uninstalled 2 packages in 7ms
Installed 2 packages in 10ms
 - fsspec==2025.12.0
 + fsspec==2025.10.0
 - huggingface-hub==0.36.0
 + huggingface-hub==1.2.3


In [32]:
# @title Creat and upload dataset to HF
HF_USERNAME = "dzungpham"
!python FontDiffusion/create_hf_dataset.py \
  --data-dir "my_dataset/train_original" \
  --repo-id dzungpham/font-diffusion-generated-data \
  --split "train_original" \
  --token {HF_TOKEN}

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data-dir "my_dataset/train" \
  --repo-id dzungpham/font-diffusion-generated-data \
  --split "train" \
  --token {HF_TOKEN}

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data-dir "my_dataset/val" \
  --repo-id dzungpham/font-diffusion-generated-data \
  --split "val" \
  --token {HF_TOKEN}


2026-01-03 05:42:19,648 | INFO | Directory structure validated successfully
2026-01-03 05:42:19,649 | INFO | Building dataset...
2026-01-03 05:42:19,775 | INFO | Loaded checkpoint: 14220 generations, 948 characters, 15 styles
Loading image pairs: 100%|██████████████████████████████████| 13.9k/13.9k [00:12<00:00, 1.13kpair/s]
2026-01-03 05:42:32,380 | INFO | Successfully loaded 14220 samples
2026-01-03 05:43:09,270 | INFO | Pushing dataset to dzungpham/font-diffusion-generated-data...
Uploading the dataset shards:   0% 0/1 [00:00<?, ? shards/s]
Map:   0% 0/14220 [00:00<?, ? examples/s]
Map:  47% 6739/14220 [00:00<00:00, 18796.38 examples/s]
Map: 100% 14220/14220 [00:00<00:00, 18936.49 examples/s]

Creating parquet from Arrow format:   0% 0/3 [00:00<?, ?ba/s]
Creating parquet from Arrow format:  33% 1/3 [00:00<00:00,  4.29ba/s]
Creating parquet from Arrow format: 100% 3/3 [00:00<00:00,  6.68ba/s]
Processing Files (0 / 0)      : |          |  0.00B /  0.00B            
New Data Upload    

In [ ]:
# import torch, gc
torch.cuda.empty_cache()
gc.collect()

In [24]:
# @title Training phase 1
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2"
import wandb

MAX_TRAIN_STEPS = 1000
!accelerate launch FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_1" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --phase_1_ckpt_dir="ckpt" \
    --report_to="wandb" \
      \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
      \
    --train_batch_size=16 \
    --gradient_accumulation_steps=1 \
    --perceptual_coefficient=0.05 \
    --offset_coefficient=0.5 \
    --max_train_steps={MAX_TRAIN_STEPS} \
    --ckpt_interval={MAX_TRAIN_STEPS // 2} \
    --log_interval=50 \
      \
    --learning_rate=1e-4 \
    --lr_scheduler="linear" \
    --lr_warmup_steps=100 \
    --drop_prob=0.1 \
    --mixed_precision="no"

Using Python 3.12.12 environment at: /usr
Resolved 13 packages in 56ms
Prepared 2 packages in 1ms
Uninstalled 2 packages in 4ms
Installed 2 packages in 6ms
 - fsspec==2025.10.0
 + fsspec==2025.12.0
 - huggingface-hub==1.2.3
 + huggingface-hub==0.36.0
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/content/FontDiffusion/src/dpm_solver/dpm_solver_pytorch.py:53: SyntaxWarning: invalid escape sequence '\h'
  The `alphas_cumprod` is the \hat{alpha_n} arrays in the notations of DDPM. Specifically, DDPMs assume that
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Load the down block  DownBlock2D
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 

In [ ]:
!ls -lr outputs/FontDiffuser

In [ ]:
# @title Training phase 2 with SCR
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2"

!wandb login
MAX_TRAIN_STEPS = 500
!accelerate launch FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_2" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --phase_2 \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_500" \
    --scr_ckpt_path="ckpt/scr_210000.pth" \
    \
    --sc_coefficient=0.05 \
    --num_neg=10 \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    \
    --train_batch_size=16 \
    --gradient_accumulation_steps=1 \
    --perceptual_coefficient=0.05 \
    --offset_coefficient=0.5 \
    --max_train_steps={MAX_TRAIN_STEPS} \
    --ckpt_interval={MAX_TRAIN_STEPS // 2} \
    --log_interval=50 \
    \
    --learning_rate=1e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=100 \
    --drop_prob=0.1 \
    --mixed_precision="no"


In [ ]:
!ls -l outputs/FontDiffuser/*/*

-rw-r--r-- 1 root root   4756580 Jan  2 16:16 outputs/FontDiffuser/global_step_1000/content_encoder.safetensors
-rw-r--r-- 1 root root  82394556 Jan  2 16:16 outputs/FontDiffuser/global_step_1000/style_encoder.safetensors
-rw-r--r-- 1 root root 402093724 Jan  2 16:16 outputs/FontDiffuser/global_step_1000/total_model.safetensors
-rw-r--r-- 1 root root 314927748 Jan  2 16:16 outputs/FontDiffuser/global_step_1000/unet.safetensors
-rw-r--r-- 1 root root   4756580 Jan  2 15:03 outputs/FontDiffuser/global_step_250/content_encoder.safetensors
-rw-r--r-- 1 root root  82394556 Jan  2 15:03 outputs/FontDiffuser/global_step_250/style_encoder.safetensors
-rw-r--r-- 1 root root 402084924 Jan  2 15:03 outputs/FontDiffuser/global_step_250/total_model.safetensors
-rw-r--r-- 1 root root 314927748 Jan  2 15:03 outputs/FontDiffuser/global_step_250/unet.safetensors
-rw-r--r-- 1 root root   4756580 Jan  2 15:59 outputs/FontDiffuser/global_step_500/content_encoder.safetensors
-rw-r--r-- 1 root root  8239455

In [ ]:
STEP = 1000
!python FontDiffusion/upload_models.py \
    --weights_dir "outputs/FontDiffuser/global_step_{STEP}" \
    --repo_id "dzungpham/font-diffusion-weights" \
    --token "{HF_TOKEN}"


PYTORCH TO SAFETENSORS CONVERTER & HF UPLOADER

VALIDATING INPUTS
✓ Weights directory: outputs/FontDiffuser/global_step_1000
✓ HF token: ********************

✓ Files to process: 10
  ⚠ content_encoder.pth (not found)
  ✓ content_encoder.safetensors (4.54 MB)
  ⚠ style_encoder.pth (not found)
  ✓ style_encoder.safetensors (78.58 MB)
  ⚠ unet.pth (not found)
  ✓ unet.safetensors (300.34 MB)
  ⚠ total_model.pth (not found)
  ✓ total_model.safetensors (383.47 MB)
  ⚠ scr.pth (not found)
  ⚠ scr.safetensors (not found)

CONVERTING .pth TO .safetensors

⚠ content_encoder.pth: Not found, skipping

⚠ style_encoder.pth: Not found, skipping

⚠ unet.pth: Not found, skipping

⚠ total_model.pth: Not found, skipping

⚠ scr.pth: Not found, skipping

----------------------------------------------------------------------
Conversion complete: 0 succeeded, 0 failed

UPLOADING TO HUGGING FACE HUB
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this functi

In [ ]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="my_dataset")
    except Exception as e:
        print(f"❌ An error occurred: {e}")